# Homework 2: Recipe Bot Error Analysis
##### By: Ishan Gore

Before starting the homework, let's first import all necessary packages and initialize our caller to Claude:

In [ ]:
# to help format LLM prompts
from textwrap import dedent
# to set a seed for sampling
import random

In [ ]:
# package for using LLM with AWS Bedrock
from anthropic import AnthropicBedrock
from claudette import models_aws, Client, Chat

model_id = "us.anthropic.claude-3-5-sonnet-20241022-v2:0"
ab = AnthropicBedrock(aws_profile="gossamer_aws_profile", aws_region="us-east-1")
client = Client(model_id, ab)
chat = Chat(cli=client)

## Part 1: Define Dimensions & Generate Initial Queries

Let's first define some dimensions that we expect to see in our user queries:

Dimensions:
- cuisine_type
    - thai
    - jamaican
    - nordic
    - kyrgz
- dietary_restrictions
    - no restrictions
    - vegetarian
    - lactose intolerant and no red meat
    - vegan and gluten free
- complexity
    - simple
    - average
    - fancy
- time constraint
    - 30 minutes
    - 1 hour
    - whole day
- serving portion
    - 1 person
    - 4 people
    - 20 people

Now let's make an LLM prompt to take those dimensions and output a cross product of tuples:

In [ ]:
DIMENSION_CREATION_PROMPT=dedent("""

    I am in need of help with generated a set of tuples to be used as test cases for a recipe chatbot. The tuples are to be generated based on various dimensions and their possible values. The dimensions are based off of user queries that we expect to see.

    Generate a list of tuples that represent the cross product (all possible combinations) of the provided dimensions and their possible values.

    The tuples should be the same length as the number of dimensions provided. The format of the tuples should be:
    (Dimension1_value, Dimension2_value, Dimension3_value, etc...)

    Below are the dimensions and their possible values:
    cuisine_type: thai, jamaican, nordic, kyrgz
    dietary_restrictions: no restrictions, vegetarian, lactose intolerant and no red meat, vegan and gluten free
    complexity: simple, average complexity, fancy
    time constraint: 30 minutes, 1 hour, whole day
    serving portion: 1 person, 4 people, 20 people

    With the given dimensions and values above, generate the requested tuples. The order of the dimensions in the tuples should be as listed above. Generate 50 tuples from the cross product that are varied and reasonable so that it represents a good sample of potential user queries.
    Each value in the tuple should be encapsulated in quotation marks "" to represent a string character.

""")
print(DIMENSION_CREATION_PROMPT)

In [ ]:
chat(DIMENSION_CREATION_PROMPT)

In [ ]:
dimension_examples = (
    ("thai", "no restrictions", "simple", "30 minutes", "4 people"),
    ("jamaican", "vegetarian", "average complexity", "1 hour", "4 people"),
    ("nordic", "vegan and gluten free", "fancy", "whole day", "20 people"),
    ("kyrgz", "lactose intolerant and no red meat", "simple", "1 hour", "4 people"),
    ("thai", "vegetarian", "fancy", "whole day", "20 people"),
    ("jamaican", "no restrictions", "simple", "30 minutes", "1 person"),
    ("nordic", "lactose intolerant and no red meat", "average complexity", "1 hour", "4 people"),
    ("kyrgz", "vegan and gluten free", "fancy", "whole day", "20 people"),
    ("thai", "no restrictions", "average complexity", "1 hour", "4 people"),
    ("jamaican", "vegetarian", "simple", "30 minutes", "4 people"),
    ("nordic", "no restrictions", "fancy", "whole day", "20 people"),
    ("kyrgz", "vegan and gluten free", "simple", "30 minutes", "1 person"),
    ("thai", "lactose intolerant and no red meat", "average complexity", "1 hour", "4 people"),
    ("jamaican", "no restrictions", "fancy", "whole day", "20 people"),
    ("nordic", "vegetarian", "simple", "30 minutes", "4 people"),
    ("kyrgz", "no restrictions", "average complexity", "1 hour", "4 people"),
    ("thai", "vegan and gluten free", "fancy", "whole day", "20 people"),
    ("jamaican", "lactose intolerant and no red meat", "simple", "30 minutes", "1 person"),
    ("nordic", "no restrictions", "average complexity", "1 hour", "4 people"),
    ("kyrgz", "vegetarian", "fancy", "whole day", "20 people"),
    ("thai", "no restrictions", "simple", "1 hour", "4 people"),
    ("jamaican", "vegan and gluten free", "average complexity", "whole day", "20 people"),
    ("nordic", "lactose intolerant and no red meat", "fancy", "30 minutes", "4 people"),
    ("kyrgz", "no restrictions", "simple", "1 hour", "1 person"),
    ("thai", "vegetarian", "average complexity", "whole day", "20 people"),
    ("jamaican", "no restrictions", "fancy", "30 minutes", "4 people"),
    ("nordic", "vegan and gluten free", "simple", "1 hour", "4 people"),
    ("kyrgz", "lactose intolerant and no red meat", "average complexity", "whole day", "20 people"),
    ("thai", "no restrictions", "fancy", "1 hour", "4 people"),
    ("jamaican", "vegetarian", "simple", "whole day", "20 people"),
    ("nordic", "no restrictions", "average complexity", "30 minutes", "1 person"),
    ("kyrgz", "vegan and gluten free", "fancy", "1 hour", "4 people"),
    ("thai", "lactose intolerant and no red meat", "simple", "whole day", "20 people"),
    ("jamaican", "no restrictions", "average complexity", "30 minutes", "4 people"),
    ("nordic", "vegetarian", "fancy", "1 hour", "4 people"),
    ("kyrgz", "no restrictions", "simple", "whole day", "20 people"),
    ("thai", "vegan and gluten free", "average complexity", "30 minutes", "1 person"),
    ("jamaican", "lactose intolerant and no red meat", "fancy", "1 hour", "4 people"),
    ("nordic", "no restrictions", "simple", "whole day", "20 people"),
    ("kyrgz", "vegetarian", "average complexity", "30 minutes", "4 people"),
    ("thai", "no restrictions", "fancy", "1 hour", "1 person"),
    ("jamaican", "vegan and gluten free", "simple", "whole day", "20 people"),
    ("nordic", "lactose intolerant and no red meat", "average complexity", "30 minutes", "4 people"),
    ("kyrgz", "no restrictions", "fancy", "1 hour", "4 people"),
    ("thai", "vegetarian", "simple", "whole day", "20 people"),
    ("jamaican", "no restrictions", "average complexity", "30 minutes", "1 person"),
    ("nordic", "vegan and gluten free", "fancy", "1 hour", "4 people"),
    ("kyrgz", "lactose intolerant and no red meat", "simple", "whole day", "20 people"),
    ("thai", "no restrictions", "average complexity", "30 minutes", "4 people"),
    ("jamaican", "vegetarian", "fancy", "1 hour", "20 people")
)

In [ ]:
random.seed(42)
dimension_samples_subset = random.sample(dimension_examples, 10)

In [ ]:
QUERY_CREATION_PROMPT = dedent("""

    I will give you a subset of the tuples you just created. For each of those tuples, generate exactly one query that represents a user asking for a recipe according to the dimensions of the corresponding tuple.
    Vary each query in terms of characteristics like (but not limited to) length, tone, spelling, and syntax to encapsulate a broad set of potential user personalities.

    Below is the subset of tuples for you to create your queries from:
    {dimension_samples_subset}

    Generate the queries.

""")
print(QUERY_CREATION_PROMPT.format(dimension_samples_subset=dimension_samples_subset))

In [ ]:
chat(QUERY_CREATION_PROMPT.format(dimension_samples_subset=dimension_samples_subset))

Below are our final natural language test queries generated to test our recipe bot:


**1. thai + no restrictions + fancy + 1 hour + 1 person**

I'm treating myself to a fancy Thai dinner tonight! Got an hour to cook something special just for me 👨‍🍳

**2. kyrgz + vegan and gluten free + fancy + whole day + 20 people**

Need help planning a large Kyrgyz feast for my wedding - looking for vegan & GF options. Have all day to prepare and need to feed 20 guests. Any traditional recipes?

**3. jamaican + vegetarian + average complexity + 1 hour + 4 people**

Can u suggest a vegetarian jamaican recipe that'll feed my family of 4? nothing too complicated, got about an hour

**4. jamaican + lactose intolerant and no red meat + simple + 30 minutes + 1 person**

Quick & easy Jamaican recipe for one? Can't have dairy or red meat, and need it done in 30 mins!

**5. kyrgz + no restrictions + average complexity + 1 hour + 4 people**

Would love to explore Kyrgyz cuisine! Looking for a medium-difficulty recipe that serves 4. We eat everything and I've got an hour to spare.

**6. nordic + vegetarian + simple + 30 minutes + 4 people**
Help!! Need a super quick vegetarian Nordic recipe for unexpected dinner guests coming in 30 mins!!! Recipe must serve 4 ppl

**7. thai + no restrictions + average complexity + 1 hour + 4 people**

Hi there! Could you recommend a nice Thai dish that would work for a family dinner? We have about an hour to cook and there's four of us.

**8. nordic + lactose intolerant and no red meat + average complexity + 1 hour + 4 people**

Looking 4 a Nordic recipe that doesn't have dairy/red meat... cooking 4 my inlaws tonight (4ppl total). Have 1hr, nothing 2 fancy plz

**9. nordic + vegetarian + fancy + 1 hour + 4 people**

I desire an elegant Nordic vegetarian dish that will impress my dinner guests this evening. We are four persons in total, and I have precisely one hour to prepare it.

**10. jamaican + no restrictions + simple + 30 minutes + 1 person**

yo! hit me up w/ a quick jamaican recipe for lunch? got like 30 mins max lol

## Part 2: Initial Error Analysis

Putting the synthetic queries generated above into a csv, we ran bulk_test.py against that csv:

python scripts/bult_test.py --csv homeworks/hw2/synthetic_queries.csv

The output is captured in homeworks/hw2/synthetic_queries_with_response.csv